# Nova Neural Network Whiteboard

This code is just me experimenting with building my own neural network, none of this has been unit tested, time or memory efficiency optimized, etc...

In [1]:
import numpy as np
from neuralnetworks import fnn, embedding

Documentation on softmax and ReLU are provided below: <br>
**[ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))** <br>
**[softmax](https://en.wikipedia.org/wiki/Softmax_function)**

In [12]:
class Nova:
    def __init__(self, hidden_dim = 125, vocab = None):
        self.hidden_dim = hidden_dim
        self.vocab = vocab
        self.prompt = None
        self.tokens = None
        self.code = None
        self.memory = []
        self.E = embedding.Embedding(hidden_dim)
        self.NN = fnn.Neural(len(vocab), embedding_instance = self.E, hidden_dim = hidden_dim)
        
    @property
    def Prompt(self):
        return self.prompt
        
    @Prompt.setter
    def Prompt(self, prompt_str):
        # PLACEHOLDER SET TOKENS VIA TOKENIZATION
        self.tokens = [prompt_str]
        self.prompt = prompt_str
        return
        
    def generateCode(self):
        if not self.tokens:
            return "please enter a prompt"
        for token in self.tokens:
            self.E.setParams(token)
        return self.NN.embedding.hmap
        #sparse attention head placeholder
        

In [13]:
nova = Nova(vocab = ["yes", "no"])

nova.Prompt = "the world is round"

nova.generateCode()

{'the world is round': 0}

In [30]:
sum([1 if v > 0 else 0 for v in e_model.M[0]])

250546